In [1]:
import os
from torch.optim import AdamW
from torch import nn
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder

C:\Users\Gumpun\AppData\Local\Temp\ipykernel_15816\196635466.py:8: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def file2list(fname):
    with open(fname, encoding='utf-8') as fp:  # Specify the encoding
        lines = fp.readlines()
        return [line.strip() for line in lines]

In [3]:
pos_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Positive.csv")
neg_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Negative.csv")
neu_text = ("C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/DataReviewProductThai/Neutrally.csv")

In [4]:
pos_df = pd.read_csv(pos_text, delimiter=',', encoding='utf-8').dropna()
neg_df = pd.read_csv(neg_text, delimiter=',', encoding='utf-8').dropna()
neu_df = pd.read_csv(neu_text, delimiter=',', encoding='utf-8').dropna()

data = pd.concat([pos_df, neg_df, neu_df])
data.sample(100)

,User,Message,Sentiment
217,Ronnarong P.,เนื้อผ้าบางเกินคาด,Neutrally
1419,0***7,ไม่มีรูลำโพงค่ะแต่ก็โอเคอยู่,Neutrally
8,v***n,บางมากคะ,Negative
1186,เ***.,สินค้าไม่ตรงปก,Negative
140,Pimpida P.,ดีมาก,Positive
...,...,...,...
414,T***.,ดี,Neutrally
834,tananya,แย่มากสั่งมาอัน อันนึงได้ยินเสียงสะท้อน อีกอัน...,Negative
479,WANIDA🤍🦋,คนละสีกะที่สั่งเลย,Negative
1317,จ***.,การทำงานของร้านค้าช้ามาก กว่าจะส่งของช้ามากๆ แย่,Negative


In [5]:
category_counts = data['Sentiment'].value_counts()
category_counts

Sentiment
Positive     1499
Negative     1496
Neutrally    1496
Name: count, dtype: int64

In [6]:
!pip install pythainlp

In [7]:
from pythainlp.tokenize import word_tokenize

In [8]:
# สร้างฟังก์ชันสำหรับการลบอักขระที่ซ้ำกันในแต่ละคำเท่านั้น
def remove_duplicate_chars(text):
    if isinstance(text, str):  # ตรวจสอบว่าข้อความไม่ใช่ NaN
        unique_words = []
        words = word_tokenize(text)
        for word in words:
            unique_word = ''
            for char in word:
                if char not in unique_word:
                    unique_word += char
            unique_words.append(unique_word)
        return ' '.join(unique_words)
    else:
        return text  # ส่งค่า NaN กลับหากเป็น NaN

# ใช้ฟังก์ชัน `remove_duplicate_chars` เพื่อ Tokenize และลบคำที่ซ้ำ
data['Message'] = data['Message'].apply(remove_duplicate_chars)
data.sample(10)

,User,Message,Sentiment
19,R***.,วัสดุ แข็ง ไป หน่อย ขนาด พอ ดี ี การ อกแ...,Positive
54,รุ่งนภา,แพ็กเจ แน่หา เมื่อ มาถึง ปลายทง ไม่ เกิด การ...,Positive
771,Ploy P.,แย่มาก ค่ะ สั่ง สอง รอบ เหมือนกั ทั้งสอ รอ...,Negative
991,อรนุช ภ.,เสื้อ ใส่ สบาย ไม่ หนา เนื้อผา ดี ส่ง ไว,Positive
651,สุภาภรณ์ ส.,สั่ง สี ดำ ตัว ได้ สีน้ำเงิ กับ เขียว เข้ม ๆ...,Negative
365,Jan S.,ตรง ที่ เป็น ผ้า มัน ไม่คอย กาย ดำ อ่ะ ค่ะ,Neutrally
1179,Jitlada P.,สั่ง กระท ใบ แต่ ไม่ มี ด้ามจับ เลย ห่วย ส...,Negative
596,*******601,คุณภาพ สินค้า ดี ครับ ตรง ตามคำสั่ง ซื้อ วัสดุ...,Negative
769,เจนจิรา จ.,สั่ง ตัว มี ตัว ที่ ชำรุด ถึง เปน ผ้า ถูกๆ แ...,Negative
1313,สาโรจน์ ย.,ยอดเี่ม กระเทียม ดอง แนะำ ร้าน นี้ เลย ของ ดีจ...,Positive


In [9]:
Message = data['Message'].values
Sentiment = data['Sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(Sentiment)
print("Encoded labels:", encoded_labels)
print("Mapping of encoded labels to original labels:")

for label, original_label in enumerate(encoder.classes_):
    print(f"{label}: {original_label}")

Encoded labels: [2 2 2 ... 1 1 1]
Mapping of encoded labels to original labels:
0: Negative
1: Neutrally
2: Positive


In [10]:
# แบ่งข้อมูลเป็นชุดฝึกและชุดทดสอบ
train_sentences, test_sentences, train_labels, test_labels = train_test_split(Message, encoded_labels, stratify=encoded_labels, test_size=0.1, random_state=42)

In [11]:
class BERTClassifier(nn.Module):
    def __init__(self, bert_model_name, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = BertModel.from_pretrained(bert_model_name)
        self.dropout = nn.Dropout(0.2)
        self.fc = nn.Linear(self.bert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        x = self.dropout(pooled_output)
        logits = self.fc(x)
        return logits

In [12]:
def train(model, data_loader, optimizer, scheduler, device):
    model.train()
    for batch in data_loader:
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device).long()  # Convert to Long
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        loss = nn.CrossEntropyLoss()(outputs, labels)
        loss.backward()
        optimizer.step()
        scheduler.step()

In [13]:
#ฟังก์ชันนี้ใช้สำหรับประเมินผลลัพธ์ของโมเดลที่ถูกฝึกสอน
def evaluate(model, data_loader, device):
    model.eval()
    predictions = []
    actual_labels = []
    with torch.no_grad():
        for batch in data_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['label'].to(device)
            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            _, preds = torch.max(outputs, dim=1)
            predictions.extend(preds.cpu().tolist())
            actual_labels.extend(labels.cpu().tolist())
    return accuracy_score(actual_labels, predictions), classification_report(actual_labels, predictions, zero_division=1)

In [14]:
 bert_model_name = 'monsoon-nlp/bert-base-thai'
 num_classes = 3
 max_length = 128
 batch_size = 32
 num_epochs = 10
 learning_rate = 2e-5

In [15]:
#คลาสที่ใช้สำหรับสร้าง Dataset สำหรับงานการจำแนกประเภทข้อความ
class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length
    def __len__(self):
        return len(self.texts)
    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        encoding = self.tokenizer(text, return_tensors='pt', max_length=self.max_length, padding='max_length', truncation=True)
        return {'input_ids': encoding['input_ids'].flatten(), 'attention_mask': encoding['attention_mask'].flatten(), 'label': torch.tensor(label)}

In [16]:
#โค้ดที่ให้มากำลังดึง tokenizer และสร้าง Dataset และ DataLoader สำหรับการฝึกและการทดสอบโมเดล
tokenizer = BertTokenizer.from_pretrained(bert_model_name)
train_dataset = TextClassificationDataset(train_sentences, train_labels, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = TextClassificationDataset(test_sentences, test_labels, tokenizer, max_length)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [18]:
model = BERTClassifier(bert_model_name, num_classes).to(device)

Some weights of the model checkpoint at monsoon-nlp/bert-base-thai were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [19]:
#การกำหนด optimizer และ scheduler ในการฝึกโมเดล BERT
optimizer = AdamW(model.parameters(), lr=learning_rate)
total_steps = len(train_dataloader) * num_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=total_steps)

In [20]:
for epoch in range(num_epochs):
        print(f"Epoch {epoch + 1}/{num_epochs}")
        train(model, train_dataloader, optimizer, scheduler, device)
        accuracy, report = evaluate(model, val_dataloader, device)
        print(f"Validation Accuracy: {accuracy:.4f}")
        print(report)

Epoch 1/10
Validation Accuracy: 0.5800
              precision    recall  f1-score   support

           0       0.60      0.59      0.59       150
           1       0.47      0.49      0.48       150
           2       0.68      0.66      0.67       150

    accuracy                           0.58       450
   macro avg       0.58      0.58      0.58       450
weighted avg       0.58      0.58      0.58       450

Epoch 2/10
Validation Accuracy: 0.6156
              precision    recall  f1-score   support

           0       0.57      0.73      0.64       150
           1       0.61      0.31      0.41       150
           2       0.67      0.81      0.73       150

    accuracy                           0.62       450
   macro avg       0.62      0.62      0.59       450
weighted avg       0.62      0.62      0.59       450

Epoch 3/10
Validation Accuracy: 0.6311
              precision    recall  f1-score   support

           0       0.55      0.87      0.68       150
           1

In [21]:
torch.save(model.state_dict(), "C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/bert_classifier.pt")

In [60]:
model.load_state_dict(torch.load('C:/Users/Gumpun/Sentiment-Analysis-System-for-Consumer-Products/bert_classifier.pt'))

<All keys matched successfully>

In [62]:
def predict_sentiment(text, model, tokenizer, device, max_length=128):
    model.eval()
    encoding = tokenizer(text, return_tensors='pt', max_length=max_length, padding='max_length', truncation=True)
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        _, preds = torch.max(outputs, dim=1)
        
    sentiment_mapping = {0: "Negative", 1: "Neutrally", 2: "Positive"}
    return sentiment_mapping[preds.item()]

In [72]:
test_text = remove_duplicate_chars("คนละสีกะที่สั่งเลย")
sentiment = predict_sentiment(test_text, model, tokenizer, device)
print(f"Predicted sentiment: {sentiment}")

Predicted sentiment: Negative


In [74]:
tokenized_text = word_tokenize(remove_duplicate_chars(test_text), keep_whitespace=False)
tokenized_text

['คนละ', 'สี', 'กะ', 'ที่', 'สั่ง', 'เลย']

In [76]:
for i in tokenized_text:
    print(i)
    test_text = remove_duplicate_chars(i)
    sentiment = predict_sentiment(test_text, model, tokenizer, device)
    print(f"Predicted sentiment: {sentiment}")
    print("----------------------")

คนละ
Predicted sentiment: Neutrally
----------------------
สี
Predicted sentiment: Neutrally
----------------------
กะ
Predicted sentiment: Negative
----------------------
ที่
Predicted sentiment: Positive
----------------------
สั่ง
Predicted sentiment: Negative
----------------------
เลย
Predicted sentiment: Negative
----------------------
